In [4]:
import numpy as np
import pandas as pd

In [7]:
import PIL
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import KFold
import copy

In [11]:
from torchsummary import summary

In [13]:

import torch
import torchvision.datasets as dset
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, SubsetRandomSampler, DataLoader

In [20]:
!pip install plotly

import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objs as go

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/df/79/c80174d711ee26ee5da55a9cc3e248f1ec7a0188b5e4d6bbbbcd09b974b0/plotly-5.17.0-py2.py3-none-any.whl.metadata
  Using cached plotly-5.17.0-py2.py3-none-any.whl.metadata (7.0 kB)
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl.metadata
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.6 MB 330.3 kB/s eta 0:00:48
   ---------------------------------------- 0.0/15.6 MB 330.3 kB/s eta 0:00:48
   ---------------------------------------- 0.0/15.6 MB 178.6 kB/s eta 0:01:28
   ---------------------------------------- 0.1/15.6 MB 280.5 kB/s eta 0:00:56
 

In [22]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Using", device, "device.")


Using cpu device.


In [23]:
import PIL.ImageOps

def to_negative(img):
    img = PIL.ImageOps.invert(img)
    return img
class Negative(object):
    """Convert image to negative.

    Args:
        

    Returns:
        PIL Image: Negative version of the input.
         
    """
    def __init__(self):
        pass
    
    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to be converted to Negative.

        Returns:
            PIL Image: Negatived image.
        """
        return to_negative(img)

In [24]:
IMG_H = 200
IMG_W = 200

In [25]:
import os
from torchvision import transforms
# Define the transformation pipeline
transform = transforms.Compose([
    transforms.Resize((IMG_H, IMG_W)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [27]:
dataset = dset.ImageFolder(
    root=r'C:\Users\Tharsigan J\Desktop\cnn\root',
    transform=transform
)

In [28]:
dataset

Dataset ImageFolder
    Number of datapoints: 746
    Root location: C:\Users\Tharsigan J\Desktop\cnn\root
    StandardTransform
Transform: Compose(
               Resize(size=(200, 200), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [29]:
# 746, divide our dataset into validation set also 746/5 -> 149.2
train_dataset, test_dataset = random_split(
    dataset=dataset, 
    lengths = [596, 150],
    generator=torch.Generator().manual_seed(42)
)

In [30]:
test_dataset

In [31]:
train_transformation = transforms.Compose([
    transforms.RandomRotation(5),
    Negative(),
    transforms.RandomAffine(
        degrees=0,
        scale=(1.1, 1.1), 
        shear=0.9
    ),
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [32]:
# transformation 
augments = copy.deepcopy(train_dataset) 
augments.dataset.transform = train_transformation 
augments

In [33]:
final_train_dataset =  torch.utils.data.ConcatDataset([train_dataset,augments])

In [34]:
#1


class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.model = nn.Sequential(
              
              nn.Conv2d(in_channels=3, out_channels=128, kernel_size=5,padding=1),  
              nn.BatchNorm2d(128),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2), 

              nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3,padding=0), 
              nn.BatchNorm2d(256),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2), 

              nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5,padding=0),
              nn.BatchNorm2d(512),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2,stride=2),
            
              nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3,padding=0), 
              nn.BatchNorm2d(1024),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2,),  

              nn.Conv2d(in_channels=1024, out_channels=2048, kernel_size=5,padding=1),
              nn.BatchNorm2d(2048),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2,stride=2),
            
            
              nn.Flatten() ,
              nn.Dropout(0.5),  
            
              nn.Linear(in_features= 32768, out_features=1024),
              nn.ReLU(),
              nn.Dropout(0.3),
              nn.Linear(in_features=1024, out_features=256),
              nn.ReLU(),
              nn.Linear(in_features=256, out_features=64),
              nn.ReLU(),
              nn.Linear(in_features=64, out_features=1)
        )
        
        
    def forward(self, x):
             return  self.model(x)


In [35]:
cnn_1 = Model1()
cnn_1 = cnn_1.to(device)
summary(cnn_1,(3,200,200),200)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1       [200, 128, 198, 198]           9,728
       BatchNorm2d-2       [200, 128, 198, 198]             256
              ReLU-3       [200, 128, 198, 198]               0
         MaxPool2d-4         [200, 128, 99, 99]               0
            Conv2d-5         [200, 256, 97, 97]         295,168
       BatchNorm2d-6         [200, 256, 97, 97]             512
              ReLU-7         [200, 256, 97, 97]               0
         MaxPool2d-8         [200, 256, 48, 48]               0
            Conv2d-9         [200, 512, 44, 44]       3,277,312
      BatchNorm2d-10         [200, 512, 44, 44]           1,024
             ReLU-11         [200, 512, 44, 44]               0
        MaxPool2d-12         [200, 512, 22, 22]               0
           Conv2d-13        [200, 1024, 20, 20]       4,719,616
      BatchNorm2d-14        [200, 1024,

c:\users\tharsigan j\appdata\local\programs\python\python38\lib\site-packages\torchsummary\torchsummary.py:93: RuntimeWarning: overflow encountered in scalar add
  total_output += np.prod(summary[layer]["output_shape"])


In [36]:
summary(cnn_1,(3,200,200),200)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1       [200, 128, 198, 198]           9,728
       BatchNorm2d-2       [200, 128, 198, 198]             256
              ReLU-3       [200, 128, 198, 198]               0
         MaxPool2d-4         [200, 128, 99, 99]               0
            Conv2d-5         [200, 256, 97, 97]         295,168
       BatchNorm2d-6         [200, 256, 97, 97]             512
              ReLU-7         [200, 256, 97, 97]               0
         MaxPool2d-8         [200, 256, 48, 48]               0
            Conv2d-9         [200, 512, 44, 44]       3,277,312
      BatchNorm2d-10         [200, 512, 44, 44]           1,024
             ReLU-11         [200, 512, 44, 44]               0
        MaxPool2d-12         [200, 512, 22, 22]               0
           Conv2d-13        [200, 1024, 20, 20]       4,719,616
      BatchNorm2d-14        [200, 1024,

In [37]:
#model2

class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.model = nn.Sequential(
              
              nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3),  
              nn.BatchNorm2d(128),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2), 

              nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3), 
              nn.BatchNorm2d(256),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2), 

              nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3),
              nn.BatchNorm2d(512),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2),  

              nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3), 
              nn.BatchNorm2d(1024),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2),
            
              nn.Conv2d(in_channels=1024, out_channels=2048, kernel_size=3),
              nn.BatchNorm2d(2048),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2),
            
              nn.Flatten() ,
              nn.Dropout(0.2),  
            
                nn.Linear(in_features= 32768, out_features=2048),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(in_features=2048, out_features=512),
                nn.ReLU(),
                nn.Linear(in_features=512, out_features=128),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=1)
        )
        
    def forward(self, x):
        return  self.model(x)


In [38]:
cnn_2 = Model2()
cnn_2 = cnn_2.to(device)
summary(cnn_2,(3,200,200),32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [32, 128, 198, 198]           3,584
       BatchNorm2d-2        [32, 128, 198, 198]             256
              ReLU-3        [32, 128, 198, 198]               0
         MaxPool2d-4          [32, 128, 99, 99]               0
            Conv2d-5          [32, 256, 97, 97]         295,168
       BatchNorm2d-6          [32, 256, 97, 97]             512
              ReLU-7          [32, 256, 97, 97]               0
         MaxPool2d-8          [32, 256, 48, 48]               0
            Conv2d-9          [32, 512, 46, 46]       1,180,160
      BatchNorm2d-10          [32, 512, 46, 46]           1,024
             ReLU-11          [32, 512, 46, 46]               0
        MaxPool2d-12          [32, 512, 23, 23]               0
           Conv2d-13         [32, 1024, 21, 21]       4,719,616
      BatchNorm2d-14         [32, 1024,

In [39]:
class Model3(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.model = nn.Sequential(
              
              nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5,padding=1),  
              nn.BatchNorm2d(64),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2), 

              nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3,padding=0), 
              nn.BatchNorm2d(128),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2), 

              nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5,padding=0),
              nn.BatchNorm2d(256),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2,stride=2),  

              nn.Conv2d(in_channels=256, out_channels=500, kernel_size=3,padding=0), 
              nn.BatchNorm2d(500),
              nn.ReLU(),
              nn.MaxPool2d(kernel_size=2,stride=2),  

#               nn.Conv2d(in_channels=1024, out_channels=2048, kernel_size=5,padding=1),
#               nn.BatchNorm2d(2048),
#               nn.ReLU(),
#               nn.MaxPool2d(kernel_size=2,stride=2),
            
            
              nn.Flatten() ,
              nn.Dropout(0.2),  
            
                nn.Linear(in_features= 50000, out_features=1024),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(in_features=1024, out_features=256),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=1)
        )
        
    def forward(self, x):
        return  self.model(x)

In [40]:
cnn_3 = Model3()
cnn_3 = cnn_3.to(device)
summary(cnn_3,(3,200,200),32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 198, 198]           4,864
       BatchNorm2d-2         [32, 64, 198, 198]             128
              ReLU-3         [32, 64, 198, 198]               0
         MaxPool2d-4           [32, 64, 99, 99]               0
            Conv2d-5          [32, 128, 97, 97]          73,856
       BatchNorm2d-6          [32, 128, 97, 97]             256
              ReLU-7          [32, 128, 97, 97]               0
         MaxPool2d-8          [32, 128, 48, 48]               0
            Conv2d-9          [32, 256, 44, 44]         819,456
      BatchNorm2d-10          [32, 256, 44, 44]             512
             ReLU-11          [32, 256, 44, 44]               0
        MaxPool2d-12          [32, 256, 22, 22]               0
           Conv2d-13          [32, 500, 20, 20]       1,152,500
      BatchNorm2d-14          [32, 500,

In [24]:
#Transfer learning models

In [47]:
import torchvision.models as models
vgg19 = models.vgg19(pretrained=True)

c:\users\tharsigan j\appdata\local\programs\python\python38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\users\tharsigan j\appdata\local\programs\python\python38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\Tharsigan J/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100.0%


In [48]:
vgg19

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [49]:
for param in vgg19.parameters():
   param.requires_grad = False

vgg19.classifier[6].in_features

4096

In [50]:
# number_features = vgg_based.classifier[6].in_features
# features = list(vgg_based.classifier.children())[:-1] # Remove last layer
# features.extend([torch.nn.Linear(number_features, len(class_names))])
# vgg_based.classifier = torch.nn.Sequential(*features)

In [51]:
vgg19.classifier[6] = nn.Sequential(
    nn.Linear(vgg19.classifier[6].in_features,512),
    nn.ReLU(),
    nn.Linear(512,1)
)

In [52]:
vgg19

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [53]:
vgg19=vgg19.to(device)
summary(vgg19,(3,200,200),32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 200, 200]           1,792
              ReLU-2         [32, 64, 200, 200]               0
            Conv2d-3         [32, 64, 200, 200]          36,928
              ReLU-4         [32, 64, 200, 200]               0
         MaxPool2d-5         [32, 64, 100, 100]               0
            Conv2d-6        [32, 128, 100, 100]          73,856
              ReLU-7        [32, 128, 100, 100]               0
            Conv2d-8        [32, 128, 100, 100]         147,584
              ReLU-9        [32, 128, 100, 100]               0
        MaxPool2d-10          [32, 128, 50, 50]               0
           Conv2d-11          [32, 256, 50, 50]         295,168
             ReLU-12          [32, 256, 50, 50]               0
           Conv2d-13          [32, 256, 50, 50]         590,080
             ReLU-14          [32, 256,

In [32]:
#training


In [33]:
# criterion = nn.BCELoss()
# optimizer = optim.Adam(cnn_1.parameters(), lr=0.001, weight_decay=12)

In [41]:
def getMetrics(conf_matrix):
    tp = conf_matrix[1,1]
    fp = conf_matrix[0,1]
    tn = conf_matrix[0,0]
    fn = conf_matrix[1,0]
    print("tp:{} | fp:{} | tn:{} | fn:{}".format(tp,fp,tn,fn))
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    f1 = 2*precision*recall/(precision + recall)
    return (precision, recall, f1, accuracy)

In [42]:
# model, images_train, images_test, device, lr=0.0001, epochs=30, batch_size=32, l2=0.00001, gamma=0.5, patience=7, folds=5

def train_model(params):
    model  = params["model"]
    images_train  = params["train"]
    images_test   = params["test"]
    device = params["device"] 
    lr     = params["lr"]
    batch_size  = params["batch_size"]
    epochs = params["epochs"]
    gamma  = params["gamma"]
    patience = params["patience"]
    folds = params["folds"]
    l2 = params["l2"]
    
    splits=KFold(
        n_splits=folds,
        shuffle=True,
        random_state=42
    )
    foldperf={
            'train_loss': [], 
            'train_acc': [], 
            'train_precision': [], 
            'train_recall': [],
            'train_f1': [], 
            'val_loss': [], 
            'val_acc': [], 
            'val_precision': [], 
            'val_recall': [], 
            'val_f1': []
    }
    
    model= nn.DataParallel(model)
    model = model.to(device)
    test_dataset = DataLoader(images_test, batch_size=batch_size, shuffle=False, num_workers=2)
    
    for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(images_train)))):
        print('Fold {}'.format(fold + 1))
        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler = SubsetRandomSampler(val_idx)
        
        train_dataset = DataLoader(images_train, batch_size=batch_size, sampler=train_sampler)
        val_dataset = DataLoader(images_train, batch_size=batch_size, sampler=val_sampler)
        train_loader = train_dataset
        val_loader = val_dataset
        test_loader = test_dataset

        nb_classes = 2
        
        history = {
            'train_loss': [], 
            'train_acc': [], 
            'train_precision': [], 
            'train_recall': [],
            'train_f1': [], 
            'val_loss': [], 
            'val_acc': [], 
            'val_precision': [], 
            'val_recall': [], 
            'val_f1': []
        }
        
        criterion = nn.BCEWithLogitsLoss()  
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2)  
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=patience, gamma=gamma)

        print("++++++++ Training ++++++++")
        for epoch in range(epochs):
            model.train()  

            train_loss = 0
            train_acc = 0
            val_loss = 0
            val_acc = 0

            confusion_matrix_train = torch.zeros(nb_classes, nb_classes)
            
            #  For each batch 
            for i, (images, labels) in enumerate(train_loader):
                print("=",end="")
                
                labels = labels.float()
                images = images.to(device)  
                labels = labels.to(device)  
                
                outputs = model(images).view(-1)  
                pred = torch.sigmoid(outputs)
                pred = torch.round(pred)
                cur_train_loss = criterion(outputs, labels)  
                cur_train_acc = (pred == labels).sum().item() / batch_size

                for t, p in zip(labels.view(-1), pred.view(-1)):
                    confusion_matrix_train[t.long(), p.long()] += 1

             
                cur_train_loss.backward()  
                optimizer.step()           
                optimizer.zero_grad()      

      
                train_loss += cur_train_loss         
            print(">")
        
            precision_train, recall_train, f1_train, accuracy_train = getMetrics(confusion_matrix_train)
            model.eval()  
            confusion_matrix_val = torch.zeros(nb_classes, nb_classes)
            with torch.no_grad():  
                for images, labels in val_loader:
                    
                    labels = labels.float()
                    images = images.to(device)
                    labels = labels.to(device)

                    outputs = model(images).view(-1)

                    # loss
                    cur_valid_loss = criterion(outputs, labels)
                    print("current lost :",cur_valid_loss )
                    val_loss += cur_valid_loss
                    # acc
                    pred = torch.sigmoid(outputs)
                    pred = torch.round(pred)


                    for t, p in zip(labels.view(-1), pred.view(-1)):
                        confusion_matrix_val[t.long(), p.long()] += 1

            precision_val, recall_val, f1_val, accuracy_val = getMetrics(confusion_matrix_val)
            scheduler.step()

            train_loss = train_loss / len(train_loader)
  
            train_acc = accuracy_train
            val_loss = val_loss / len(val_loader)
   
            val_acc = accuracy_val

            print(f"Epoch:{epoch + 1} / {epochs},lr: {optimizer.param_groups[0]['lr']:.5f} train loss:{train_loss:.5f},train acc: {train_acc:.5f},valid loss:{val_loss:.5f}, valid acc:{val_acc:.5f}")

            history['train_loss'].append(train_loss)
            history['train_acc'].append(train_acc)
            history['train_precision'].append(precision_train)
            history['train_recall'].append(recall_train)
            history['train_f1'].append(f1_train)
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)
            history['val_precision'].append(precision_val)
            history['val_recall'].append(recall_val)
            history['val_f1'].append(f1_val)
        # for each epoch mean in one fold assign to single value
        for key in history.keys():
            foldperf[key].append(np.mean(torch.tensor(history[key], device = 'cpu').numpy()))


    print('\n\nPerformance of {} fold cross validation'.format(folds))
    print("average train results")
    print("loss      :",np.mean(foldperf["train_loss"]))
    print("acc       :",np.mean(foldperf["train_acc"]))
    print("precision :",np.mean(foldperf["train_precision"]))
    print("recall    :",np.mean(foldperf["train_recall"]))
    print("f1 score  :",np.mean(foldperf["train_f1"]))
    
    
    
    print("average val results")
    print("loss      :",np.mean(foldperf["val_loss"]))
    print("acc       :",np.mean(foldperf["val_acc"]))
    print("precision :",np.mean(foldperf["val_precision"]))
    print("recall    :",np.mean(foldperf["val_recall"]))
    print("f1 score  :",np.mean(foldperf["val_f1"]))
    
    test_acc = 0
    confusion_matrix_test = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
        for images, labels in test_loader:
            labels = labels.float()
            images, labels = images.to(device), labels.to(device)

            
            outputs = model(images)

           
            pred = torch.sigmoid(outputs)
            pred = torch.round(pred)

            test_acc += (pred == labels).sum().item()
            for t, p in zip(labels.view(-1), pred.view(-1)):
                confusion_matrix_test[t.long(), p.long()] += 1

    precision_test, recall_test, f1_test, accuracy_test = getMetrics(confusion_matrix_test)
    
    print(f'\n\nTest Accuracy:', accuracy_test, 'Test Precision:', precision_test, 'Test Recall:', recall_test, 'Test F1:', f1_test)
#     print('Train', confusion_matrix_train)
#     print('Val', confusion_matrix_val)
#     print('Test', confusion_matrix_test)

    return foldperf

In [43]:
def ploting(hist,folds):
    
    fig = make_subplots(rows=1, cols=2,subplot_titles=['loss','accuracy'])
    fig.add_trace(go.Scatter(x=[*range(1,folds+1)], y=hist["train_loss"],name='train loss'),row=1, col=1)
    fig.add_trace(go.Scatter(x=[*range(1,folds+1)], y=hist["val_loss"],name='val loss'),row=1, col=1)
    fig.add_trace(go.Scatter(x=[*range(1,folds+1)], y=hist["train_acc"],name='train acc'),row=1, col=2)
    fig.add_trace(go.Scatter(x=[*range(1,folds+1)], y=hist["val_acc"],name='val acc'),row=1, col=2)
    fig.update_layout(template='plotly_white');fig.update_layout(margin={"r":0,"t":60,"l":0,"b":0},height=300)
    fig.show()

In [44]:
#train cnn model 3

In [45]:
params3 = {
    "model" :cnn_3,
    "train":final_train_dataset,
    "test":test_dataset,
    "device":device ,
    "lr" :0.0002,
    "batch_size":32,
    "epochs":10,
    "gamma": 0.5 ,
    "patience": 7,
    "folds": 5,
    "l2":0.09
}

In [46]:
hist1 = train_model(params3)

Fold 1
++++++++ Training ++++++++
==============================>
tp:326.0 | fp:237.0 | tn:200.0 | fn:190.0
current lost : tensor(0.7536)
current lost : tensor(0.7739)
current lost : tensor(0.7167)
current lost : tensor(0.7856)
current lost : tensor(0.6870)
current lost : tensor(0.6871)
current lost : tensor(0.7405)
current lost : tensor(0.8746)
tp:116.0 | fp:123.0 | tn:0.0 | fn:0.0
Epoch:1 / 10,lr: 0.00020 train loss:0.69451,train acc: 0.55194,valid loss:0.75238, valid acc:0.48536
==============================>
tp:403.0 | fp:247.0 | tn:190.0 | fn:113.0
current lost : tensor(0.6595)
current lost : tensor(0.6879)
current lost : tensor(0.6830)
current lost : tensor(0.6279)
current lost : tensor(0.6669)
current lost : tensor(0.6194)
current lost : tensor(0.6237)
current lost : tensor(0.8103)
tp:43.0 | fp:24.0 | tn:99.0 | fn:73.0
Epoch:2 / 10,lr: 0.00020 train loss:0.65667,train acc: 0.62225,valid loss:0.67232, valid acc:0.59414
==============================>
tp:361.0 | fp:186.0 | tn:251

In [54]:
params4 = {
    "model" :vgg19,
    "train":final_train_dataset,
    "test":test_dataset,
    "device":device ,
    "lr" :0.02,
    "batch_size":32,
    "epochs":10,
    "gamma": 0.5 ,
    "patience": 7,
    "folds": 5,
    "l2":0.09
}

In [55]:
hist2 = train_model(params4)

Fold 1
++++++++ Training ++++++++
==============================>
tp:294.0 | fp:201.0 | tn:236.0 | fn:222.0
current lost : tensor(0.8969)
current lost : tensor(1.0963)
current lost : tensor(1.3297)
current lost : tensor(1.4531)
current lost : tensor(1.6167)
current lost : tensor(1.2772)
current lost : tensor(0.7358)
current lost : tensor(1.4011)
tp:115.0 | fp:123.0 | tn:0.0 | fn:1.0
Epoch:1 / 10,lr: 0.02000 train loss:3.36917,train acc: 0.55614,valid loss:1.22584, valid acc:0.48117
==============================>
tp:333.0 | fp:222.0 | tn:215.0 | fn:183.0
current lost : tensor(0.6684)
current lost : tensor(0.7867)
current lost : tensor(0.8512)
current lost : tensor(0.9142)
current lost : tensor(0.7176)
current lost : tensor(0.9038)
current lost : tensor(1.2136)
current lost : tensor(0.9176)
tp:88.0 | fp:82.0 | tn:41.0 | fn:28.0
Epoch:2 / 10,lr: 0.02000 train loss:1.01163,train acc: 0.57503,valid loss:0.87164, valid acc:0.53975
==============================>
tp:321.0 | fp:212.0 | tn:225